In [19]:
import pickle

with open('cds_structure_features_dic_1800.pkl', 'rb') as f:    
    cds_3d_features_dic = pickle.load(f)

In [24]:
intersection = set(feature_dic.keys()).intersection(reduced_feature_dic.keys(), cds_3d_features_dic.keys())

common_list = list(intersection)

In [25]:
len(common_list)

17262

Noticed that this common list is not common list in SRDP. Due to many sequence cannot be processed by 5-mer and 3-mer.\
This method loss more sample than SRDP

In [27]:
# SR as '1' sample
import pandas as pd
SR = {}
# Load spreadsheet
xl = pd.ExcelFile('all_cancer_pair.xlsx')

df = xl.parse(xl.sheet_names[0], skiprows=3)

for index, row in df.iterrows():
# Use the first and second column values as a key, and set the value as 1
    SR[row[0], row[1]] = 1

    
df = xl.parse(xl.sheet_names[2], skiprows=2)

for index, row in df.iterrows():
# Use the first and second column values as a key, and set the value as 1
    SR[row[0], row[1]] = 1

print(len(SR))

/data1/xpgeng/.conda/envs/pytorch/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


3000


/data1/xpgeng/.conda/envs/pytorch/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [28]:
nocds_SR = []
for k,v in SR.items():
    if k[0] not in common_list or k[1] not in common_list:
        nocds_SR.append(k)
for i in nocds_SR:
    del SR[i]
print(len(SR))

2018


In [29]:
# Non-SR as '0' sample
from Bio import SeqIO

def extract_gene_symbol(description):
    parts = description.split(' ')
    for part in parts:
        if part.startswith('gene_symbol:'):
            return part.split(':')[1]
    return None

fasta_file = "Homocdsall.fasta"  

cds_dic = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    gene_symbol = extract_gene_symbol(record.description)
    cds_dic[gene_symbol] = str(record.seq)
    
cds_gene_list = []
for k,v in cds_dic.items():       
        cds_gene_list.append(k)
cds_gene_list = list(set(cds_gene_list))
print(len(cds_gene_list))

import random

Non_SR = {}
random.seed(127)
for i in range(20180-2018):
    a = random.randint(0, 17261)
    b = random.randint(0, 17261)
    while a == b:
        b = random.randint(0, 17261)
    if (common_list[a], common_list[b]) not in SR:
        
        Non_SR[common_list[a], common_list[b]] = 0
    
print(len(Non_SR))

19853
18161


#### DNA 5-mer 

In [3]:
from Bio import SeqIO

def extract_gene_symbol(description):
    parts = description.split(' ')
    for part in parts:
        if part.startswith('gene_symbol:'):
            return part.split(':')[1]
    return None  

fasta_file = "Homocdsall.fasta"  

cds_dic = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    gene_symbol = extract_gene_symbol(record.description)
    cds_dic[gene_symbol] = str(record.seq)
    
# Copy the dictionary to avoid issues with changing the dict size during iteration
cds_dic_copy = cds_dic.copy()

for gene_name, sequence in cds_dic_copy.items():
    for char in sequence:
        if char not in 'ATGC':
            del cds_dic[gene_name]  # Remove this gene from the original dict
            break  # No need to check the rest of the sequence

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# Function to generate k-mers
def generate_kmers(sequence, k):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# Generate 5-mers for each sequence in the dictionary
k = 5
kmers_dic = {gene_name: ' '.join(generate_kmers(seq, k)) for gene_name,
             seq in cds_dic.items()}

# Convert k-mers to feature vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(kmers_dic.values())

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to generate k-mers
def generate_kmers(sequence, k):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]


# Generate 5-mers for each sequence in the dictionary
k = 5
kmers_dic = {gene_name: ' '.join(generate_kmers(seq, k)) for gene_name, seq in cds_dic.items()}

# Convert k-mers to feature vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(kmers_dic.values())

# Create a dictionary with gene names as keys and feature vectors as values
feature_dic = {gene_name: X[i].toarray()[0] for i, gene_name in enumerate(kmers_dic.keys())}

#### Protein 3-mer PCA 

In [6]:
from Bio import SeqIO

def extract_gene_symbol(description):
    parts = description.split(' ')
    for part in parts:
        if part.startswith('gene_symbol:'):
            return part.split(':')[1]
    return None  

fasta_file = "Homopepall.fasta" 

pep_dic = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    gene_symbol = extract_gene_symbol(record.description)
    pep_dic[gene_symbol] = str(record.seq)

In [7]:
# Copy the dictionary to avoid issues with changing the dict size during iteration
pep_dic_copy = pep_dic.copy()
print(len(pep_dic))
for gene_name, sequence in pep_dic_copy.items():
    for char in sequence:
        if char not in 'ACDEFGHIKLMNPQRSTVWY':
            del pep_dic[gene_name]  # Remove this gene from the original dict
            break  # No need to check the rest of the sequence
            
print(len(pep_dic))

19853
17511


In [8]:
from collections import Counter
from itertools import product
from sklearn.decomposition import PCA
import numpy as np

# Generate all possible 3-mers
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
all_3mers = [''.join(p) for p in product(amino_acids, repeat=3)]

def get_3mer_composition(protein_sequence):
    # Generate 3-mers from the protein sequence
    sequence_3mers = [protein_sequence[i:i+3] for i in range(len(protein_sequence) - 2)]
    
    # Count the frequency of each 3-mer
    counter = Counter(sequence_3mers)
    
    # Create a dictionary with all 3-mers and their frequencies
    composition = {k: counter.get(k, 0) for k in all_3mers}
    
    return composition

# Compute the 3-mer composition for each protein
tri_peptide_composition_dic = {protein_name: get_3mer_composition(seq) for protein_name, seq in pep_dic.items()}

# Create a numpy array of the feature vectors
feature_matrix = np.array([list(dic.values()) for dic in tri_peptide_composition_dic.values()])

# Apply PCA to reduce dimensionality to 1000
pca = PCA(n_components=1000)
reduced_feature_matrix = pca.fit_transform(feature_matrix)

# Map the reduced feature vectors back to the protein names
reduced_feature_dic = {protein_name: reduced_feature_matrix[i] for i, protein_name in enumerate(tri_peptide_composition_dic.keys())}

#### test performance DNA + Protein Sequence

In [43]:
X = []
y = []
from tqdm import tqdm
import numpy as np
for gene_pair, label in tqdm(SR.items()):
    try:       
        a = feature_dic[gene_pair[0]]
        b = feature_dic[gene_pair[1]]
        c = reduced_feature_dic[gene_pair[0]]
        d = reduced_feature_dic[gene_pair[1]]
        e = cds_3d_features_dic[gene_pair[0]]
        f = cds_3d_features_dic[gene_pair[1]]
                
        X.append(np.concatenate([a,b,c,d,e,f]))
        y.append(label)        
    except:
        print(gene_pair)
        
for gene_pair, label in tqdm(Non_SR.items()):
    try:       
        a = feature_dic[gene_pair[0]]
        b = feature_dic[gene_pair[1]]
        c = reduced_feature_dic[gene_pair[0]]
        d = reduced_feature_dic[gene_pair[1]]
        e = cds_3d_features_dic[gene_pair[0]]
        f = cds_3d_features_dic[gene_pair[1]]
                
        X.append(np.concatenate([a,b,c,d,e,f]))
        y.append(label)        
    except:
        print(gene_pair)

100%|█████████████████████████████████████████████████████| 18161/18161 [00:08<00:00, 2147.93it/s]


In [38]:
from sklearn.metrics import confusion_matrix

Gaussian Naive Bayes

In [39]:
import random
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

# 合并X和y
data = list(zip(X, y))

# 打乱数据
random.shuffle(data)

# 将数据分成五部分
n = len(data) // 5
data_splits = [data[i * n:(i + 1) * n] for i in range(5)]

# 保存所有的AUC
auc_scores = []

# 初始化高斯朴素贝叶斯模型
gnb = GaussianNB()

# 对每个分割，使用它作为测试集，其余的作为训练集
for i in range(5):
    print(i)
    test_data = data_splits[i]
    train_data = [item for sublist in data_splits[:i] + data_splits[i + 1:] for item in sublist]

    # 分离X和y
    X_test, y_test = zip(*test_data)
    X_train, y_train = zip(*train_data)

    # 训练模型
    gnb.fit(X_train, y_train)

    # 在测试集上预测
    y_pred = gnb.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    
    # 计算并保存AUC
    auc = roc_auc_score(y_test, y_pred)
    auc_scores.append(auc)

# 计算平均AUC
average_auc = sum(auc_scores) / len(auc_scores)
print("Average AUC:", average_auc)

0
[[2530 1114]
 [ 175  216]]
1
[[2469 1165]
 [ 200  201]]
2
[[2509 1157]
 [ 168  201]]
3
[[2480 1113]
 [ 187  255]]
4
[[2381 1239]
 [ 204  211]]
Average AUC: 0.6089820391442514


Gradient Boosting

In [40]:
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

# 合并X和y
data = list(zip(X, y))

# 打乱数据
random.shuffle(data)

# 将数据分成五部分
n = len(data) // 5
data_splits = [data[i * n:(i + 1) * n] for i in range(5)]

# 保存所有的AUC
auc_scores = []

# 初始化梯度提升模型
gb = GradientBoostingClassifier()

# 对每个分割，使用它作为测试集，其余的作为训练集
for i in range(5):
    print(i)
    test_data = data_splits[i]
    train_data = [item for sublist in data_splits[:i] + data_splits[i + 1:] for item in sublist]

    # 分离X和y
    X_test, y_test = zip(*test_data)
    X_train, y_train = zip(*train_data)

    # 训练模型
    gb.fit(X_train, y_train)

    # 在测试集上预测
    y_pred = gb.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

    # 计算并保存AUC
    auc = roc_auc_score(y_test, y_pred)
    auc_scores.append(auc)

# 计算平均AUC
average_auc = sum(auc_scores) / len(auc_scores)
print("Average AUC:", average_auc)

0
[[3623    6]
 [ 285  121]]
1
[[3650    5]
 [ 259  121]]
2
[[3629    8]
 [ 295  103]]
3
[[3636    3]
 [ 279  117]]
4
[[3591    8]
 [ 331  105]]
Average AUC: 0.6403256618519786


Random Forest

In [41]:
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# 合并X和y
data = list(zip(X, y))

# 打乱数据
random.shuffle(data)

# 将数据分成五部分
n = len(data) // 5
data_splits = [data[i * n:(i + 1) * n] for i in range(5)]

# 保存所有的AUC
auc_scores = []

# 初始化随机森林模型
rf = RandomForestClassifier()

# 对每个分割，使用它作为测试集，其余的作为训练集
for i in range(5):
    print(i)
    test_data = data_splits[i]
    train_data = [item for sublist in data_splits[:i] + data_splits[i + 1:] for item in sublist]

    # 分离X和y
    X_test, y_test = zip(*test_data)
    X_train, y_train = zip(*train_data)

    # 训练模型
    rf.fit(X_train, y_train)

    # 在测试集上预测
    y_pred = rf.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

    # 计算并保存AUC
    auc = roc_auc_score(y_test, y_pred)
    auc_scores.append(auc)

# 计算平均AUC
average_auc = sum(auc_scores) / len(auc_scores)
print("Average AUC:", average_auc)

0
[[3609   14]
 [ 212  200]]
1
[[3623   21]
 [ 200  191]]
2
[[3590   19]
 [ 217  209]]
3
[[3622   17]
 [ 203  193]]
4
[[3630   12]
 [ 192  201]]
Average AUC: 0.7440504168402348


SVM

In [44]:
import random
from sklearn import svm
from sklearn.metrics import roc_auc_score, confusion_matrix

# Merge X and y
data = list(zip(X, y))

# Shuffle data
random.shuffle(data)

# Split data into five parts
n = len(data) // 5
data_splits = [data[i * n:(i + 1) * n] for i in range(5)]

# Store all AUC scores
auc_scores = []

# Initialize SVM model
svc = svm.SVC(probability=True)

# For each split, use it as a test set, the rest as a training set
for i in range(5):
    print(f"Processing fold {i+1}")
    test_data = data_splits[i]
    train_data = [item for sublist in data_splits[:i] + data_splits[i + 1:] for item in sublist]

    # Separate X and y
    X_test, y_test = zip(*test_data)
    X_train, y_train = zip(*train_data)

    # Train model
    svc.fit(X_train, y_train)

    # Predict on test set
    y_pred_prob = svc.predict_proba(X_test)[:, 1]
    
    # Convert probabilities to class labels
    y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

    # Calculate and save AUC
    auc = roc_auc_score(y_test, y_pred_prob)
    auc_scores.append(auc)

# Calculate average AUC
average_auc = sum(auc_scores) / len(auc_scores)
print("Average AUC:", average_auc)

Processing fold 1
[[3594   19]
 [ 237  185]]
Processing fold 2
[[3598   21]
 [ 221  195]]
Processing fold 3
[[3621   29]
 [ 231  154]]
Processing fold 4
[[3632   25]
 [ 210  168]]
Processing fold 5
[[3594   24]
 [ 226  191]]
Average AUC: 0.8672551815543038
